#### Input / Output code

taken from https://github.com/akoreman/Restricted-Boltzmann-Machine-Generative-Ising-Model/tree/main/Python%20code

In [1]:
# -*- coding: utf-8 -*-
# Import necessary libraries
import csv as csv  # For reading and handling CSV files
import numpy as np  # For numerical operations, particularly on arrays
import matplotlib.pyplot as plt  # For plotting and visualization

def ReadCSV(fileLocation):
    '''
    Function to read the CSV data files.
    Inputs:
        - fileLocation (string): Path to the CSV file containing data.
    Outputs:
        - array (nparray): A numpy array of dimensions (#objects, length of each vector).
    '''
    with open(fileLocation, 'r') as csvFile:  # Open the file at the given location in read mode
        array = []  # Initialize an empty list to store rows of the dataset
        reader = csv.reader(csvFile)  # Create a CSV reader object to iterate through the file

        for row in reader:  # Loop through each row in the CSV file
            array.append(row)  # Append the row (as a list) to the array

        return np.array(array).astype(np.double)  # Convert the list of lists to a numpy array with type double

def PlotLattice(array, size, save=False, fileName=""):
    '''
    Function to visualize a 2D lattice structure as a heatmap.
    Inputs:
        - array (list): Flattened 1D array representing a 2D lattice structure.
        - size (int): The dimension (width and height) of the square lattice.
        - save (bool, optional): If True, saves the plot as a PDF file. Default is False.
        - fileName (string, optional): Name of the file to save the plot (if save is True). Default is an empty string.
    Outputs:
        - None (plots a heatmap of the lattice).
    '''
    # Reshape the 1D array into a 2D square array of dimensions (size x size)
    plt.imshow(array.reshape((size, size)), cmap='Greys', interpolation='nearest')
    if save:  # Check if the save option is enabled
        plt.savefig(fileName + ".pdf")  # Save the plot as a PDF with the specified filename
    plt.show()  # Display the heatmap


In [2]:
from google.colab import drive
drive.mount('/content/drive')


MessageError: Error: credential propagation was unsuccessful

#### Generating training data

produced by modifying the 'montcarlo2d' code found at https://github.com/ising-model/ising-model-python/blob/main/main.py



In [ ]:
import numpy as np
import pandas as pd

import numba
import numpy as np
import random
import matplotlib.pyplot as plt
import csv


# Metropolis-Hastings update function optimized with Numba
@numba.jit(nopython=True)
def _metropolis(spin, L, beta):
    for _ in range(L ** 2):  # Each site gets a chance to flip
        x, y = random.randint(0, L - 1), random.randint(0, L - 1)
        s = spin[x, y]

        # Periodic boundary conditions
        xpp, ypp = (x + 1) % L, (y + 1) % L
        xnn, ynn = (x - 1) % L, (y - 1) % L
        R = spin[xpp, y] + spin[x, ypp] + spin[xnn, y] + spin[x, ynn]

        # Energy change if spin flips
        dH = 2 * s * R
        if dH < 0 or np.random.rand() < np.exp(-beta * dH):
            spin[x, y] = -s


class MonteCarlo2D:
    def __init__(self, L, eqstep, mcstep):
        self.L = L
        self.eqstep = eqstep
        self.mcstep = mcstep
        self.area = L ** 2

    def _init_spin(self):
        """Initialize lattice randomly with spins ±1"""
        return 2 * np.random.randint(2, size=(self.L, self.L)) - 1

    def _calc_energy(self, spin):
        """Compute energy per spin"""
        R = np.roll(spin, 1, axis=0) + np.roll(spin, -1, axis=0) + np.roll(spin, 1, axis=1) + np.roll(spin, -1, axis=1)
        return np.sum(-R * spin) / (4 * self.area)

    def _calc_magnetization(self, spin):
        """Compute magnetization per spin"""
        return np.abs(np.sum(spin)) / self.area

    def simulate(self, beta):
        """Run Monte Carlo simulation and return observables"""
        E1, M1, E2, M2 = 0, 0, 0, 0
        spin = self._init_spin()

        # Equilibration phase
        for _ in range(self.eqstep):
            _metropolis(spin, self.L, beta)

        # Monte Carlo sampling
        configurations = []  # List to store lattice configurations
        for _ in range(self.mcstep):
            _metropolis(spin, self.L, beta)
            E = self._calc_energy(spin)
            M = self._calc_magnetization(spin)
            E1 += E / self.mcstep
            M1 += M / self.mcstep
            E2 += E ** 2 / self.mcstep
            M2 += M ** 2 / self.mcstep

            # Store flattened lattice configuration
            configurations.append(spin.flatten())

        C = (E2 - E1 * E1) * beta ** 2 * self.area  # Heat capacity
        X = (M2 - M1 * M1) * beta * self.area  # Magnetic susceptibility

        return configurations, E1, M1, C, X


def plot_results(temperatures, energies, magnetizations, heat_caps, susceptibilities):
    """Plot the observables vs. temperature."""
    plt.figure(figsize=(12, 8))

    plt.subplot(2, 2, 1)
    plt.errorbar(temperatures, energies, label="Energy per spin", fmt='kx', markersize=1)
    plt.xlabel("Temperature (T)")
    plt.ylabel("⟨E⟩")
    plt.legend()

    plt.subplot(2, 2, 2)
    plt.errorbar(temperatures, magnetizations, label="Magnetization per spin", fmt='kx', markersize=1)
    plt.xlabel("Temperature (T)")
    plt.ylabel("⟨M⟩")
    plt.legend()

    plt.subplot(2, 2, 3)
    plt.errorbar(temperatures, heat_caps, label="Heat Capacity", fmt='kx', markersize=1)
    plt.xlabel("Temperature (T)")
    plt.ylabel("C")
    plt.legend()

    plt.subplot(2, 2, 4)
    plt.errorbar(temperatures, susceptibilities, label="Magnetic Susceptibility", fmt='kx', markersize=1)
    plt.xlabel("Temperature (T)")
    plt.ylabel("χ")
    plt.legend()

    plt.tight_layout()
    plt.show()


def save_to_csv(lattice_configs, temperature_labels, lattice_file, temperature_file):
    # Convert lists to numpy arrays
    lattice_configs_array = np.array(lattice_configs)
    temperature_labels_array = np.array(temperature_labels)

    # Create DataFrames
    lattice_df = pd.DataFrame(lattice_configs_array)
    temperature_df = pd.DataFrame(temperature_labels_array)

    # Save to CSV in the specified location
    lattice_df.to_csv(lattice_file, index=False, header=False)
    temperature_df.to_csv(temperature_file, index=False, header=False)

if __name__ == "__main__":
    # Simulation Parameters
    L = 16  # Lattice size (16x16)
    eqstep = 1000  # Equilibration steps
    mcstep = 25  # Monte Carlo steps per temperature (25 lattice configurations per temperature)
    temperatures = np.linspace(1.0, 4.0, 300)  # Temperatures from 1.0°C to 4.0°C (300 points)

    # Arrays to store observables
    energies, magnetizations, heat_caps, susceptibilities = [], [], [], []
    lattice_configs = []  # List to store all lattice configurations
    temperature_labels = []  # List to store corresponding temperatures

    mc_simulator = MonteCarlo2D(L, eqstep, mcstep)

    for T in temperatures:
        beta = 1.0 / T
        #print(f"Simulating T = {T:.2f} ...")
        configs, E, M, C, X = mc_simulator.simulate(beta)
        lattice_configs.extend(configs)  # Append new configurations
        temperature_labels.extend([T] * len(configs))  # Append corresponding temperatures

        # Append observables
        energies.append(E)
        magnetizations.append(M)
        heat_caps.append(C)
        susceptibilities.append(X)

    # Specify file paths for saving
    lattice_file = '/content/drive/MyDrive/UGA/Ising_model_data/LatticeConfigurations.csv'
    temperature_file = '/content/drive/MyDrive/UGA/Ising_model_data/TemperatureLabels.csv'

    # Save the data to CSV files
    save_to_csv(lattice_configs, temperature_labels, lattice_file, temperature_file)

    # Plot the results
    plot_results(temperatures, energies, magnetizations, heat_caps, susceptibilities)


#### Training the RBM

taken from taken from https://github.com/akoreman/Restricted-Boltzmann-Machine-Generative-Ising-Model/tree/main/Python%20code



In [ ]:
from sklearn.neural_network import BernoulliRBM  # For implementing Restricted Boltzmann Machines (RBM)

'''
Function to calculate the total energy of a lattice.
The lattice is reshaped into 2D and periodic boundary conditions are applied.
'''
def LatticeEnergy(lattice, size):
    Energy = 0
    lattice = (2 * lattice) - 1  # Convert {0, 1} back to {-1, 1} for calculations
    lattice = np.reshape(lattice, (size, size))  # Reshape the lattice to 2D

    # Iterate over all lattice sites and calculate interaction energy with neighbors
    for i in range(size):
        for j in range(size):
            Energy += -1 * lattice[i, j] * (
                lattice[(i + 1) % size, j] + lattice[(i - 1) % size, j] + lattice[i, (j + 1) % size] + lattice[i, (j - 1) % size]
            )

    return Energy

'''
Function to calculate the total magnetization of a lattice.
The magnetization is computed as the sum of all spins.
'''
def LatticeMagnetisation(lattice, size):
    lattice = (2 * lattice) - 1  # Convert {0, 1} back to {-1, 1}
    return np.sum(lattice)

temperatureLabels = ReadCSV('/content/drive/MyDrive/UGA/TemperatureLabels.csv')  # Load temperature labels corresponding to each lattice configuration
latticesTotal = ReadCSV('/content/drive/MyDrive/UGA/LatticeConfigurations.csv')

# Calculate lattice size based on the length of one flattened lattice vector
latticeSize = int(len(latticesTotal[0]) ** (1 / 2))
latticesTotal = (latticesTotal + 1) / 2  # Convert from {-1, 1} to {0, 1}

# Verify the shapes of the data
print(f'Lattice configurations shape: {latticesTotal.shape}')
print(f'Temperature labels shape: {temperatureLabels.shape}')

# Extract unique temperature values and determine how many samples were taken per temperature
tempList = np.unique(temperatureLabels, return_counts=True)[0]  # List of unique temperature values
print(tempList.shape)
nItr = np.unique(temperatureLabels, return_counts=True)[1][0]  # Number of samples (iterations) per temperature

'''
Reshape the lattice configurations and temperature labels into a structured format for processing.
Each temperature's configurations are grouped together for easier RBM training and analysis.
'''
latticesTotal = np.reshape(latticesTotal, (len(tempList), nItr, latticeSize ** 2))  # Reshape lattice data by temperature
temperatureLabels = np.reshape(temperatureLabels, (len(tempList), nItr))  # Reshape labels to match the new structure

# Initialize lists to store observables (magnetization, energy) and their variances
magnetisationVariance = []  # Variance of magnetization for each temperature
energyVariance = []  # Variance of energy for each temperature
magnetisation = []  # Average magnetization per spin for each temperature
energy = []  # Average energy per spin for each temperature

'''
Train an RBM for each temperature and compute physical observables using generated samples.
This process is repeated for different numbers of hidden nodes in the RBM.
'''
numberHiddenNodes = [8, 16, 256]  # Different RBM configurations with varying numbers of hidden nodes
numberHiddenNodes = [3]  # Different RBM configurations with varying numbers of hidden nodes


for h in range(len(numberHiddenNodes)):
    # Temporary storage for observables corresponding to this RBM configuration
    magnetisationOneTemperature = []
    energyOneTemperature = []
    magnetisationVarianceOneTemperature = []
    energyVarianceOneTemperature = []
    temperatureCounter = 0  # Index to track temperature in loops

    for latticeOneTemperature in latticesTotal:
        temperature = temperatureLabels[temperatureCounter, 0]  # Extract current temperature
        print("Temperature: " + str(temperature))  # Display the temperature being processed

        # Train an RBM on the lattice configurations at this temperature
        rbm = BernoulliRBM(n_components=numberHiddenNodes[h], learning_rate=0.001, verbose=False, n_iter=100, batch_size=10)
        rbm.fit(latticeOneTemperature)  # Fit the RBM to the training data

        '''
        # Visualize the trained RBM's weight matrix (hidden-visible connections)
        plt.imshow(rbm.components_)
        plt.ylabel("Hidden nodes")
        plt.xlabel("Visible nodes")
        plt.savefig("weightsTemp" + str(temperature) + "Nh" + str(numberHiddenNodes[h]) + ".pdf")
        plt.show()
        '''

        # Generate new samples using Gibbs sampling and compute observables
        magnetisationSampleList = []  # Store magnetization values from generated samples
        energySampleList = []  # Store energy values from generated samples

        randomlattice = np.random.randint(2, size=latticeSize ** 2)  # Initialize a random lattice configuration

        equilibrationTime = 1000  # Number of steps before collecting samples (to reach equilibrium)
        numberSamples = 1000  # Number of samples to generate after equilibration

        # Perform Gibbs sampling to generate new spin configurations
        for i in range(equilibrationTime * latticeSize ** 2 + numberSamples * latticeSize ** 2):
            lattice = rbm.gibbs(randomlattice)  # Generate a new lattice configuration
            # After equilibration, collect samples at intervals of latticeSize^2
            if i > equilibrationTime * latticeSize ** 2 and i % latticeSize ** 2 == 0:
                magnetisationSampleList.append(LatticeMagnetisation(lattice, latticeSize) / latticeSize ** 2)  # Normalize by lattice size
                energySampleList.append(LatticeEnergy(lattice, latticeSize))  # Compute energy

        # Compute the average and variance of magnetization and energy
        magnetisationOneTemperature.append(np.average(magnetisationSampleList))  # Mean magnetization
        energyOneTemperature.append(np.average(energySampleList) / latticeSize ** 2)  # Mean energy per spin

        magnetisationVarianceOneTemperature.append((latticeSize ** 2 / temperature) * np.var(magnetisationSampleList))  # Magnetization variance (related to susceptibility)
        energyVarianceOneTemperature.append((1 / (temperature ** 2 * latticeSize ** 2)) * np.var(energySampleList))  # Energy variance (related to specific heat)

        temperatureCounter += 1  # Move to the next temperature

    # Store the computed observables for this RBM configuration
    magnetisation.append(magnetisationOneTemperature)
    energy.append(energyOneTemperature)

    magnetisationVariance.append(magnetisationVarianceOneTemperature)
    energyVariance.append(energyVarianceOneTemperature)

# Define colors for plotting different RBM configurations
colours = ['ro', 'go', 'bo']
colours = ['go']

# Plot specific heat (energy variance) vs temperature
for i in range(len(numberHiddenNodes)):
    plt.plot(tempList, energyVariance[i], colours[i])
plt.ylabel("c [dimensionless units]")  # Specific heat label
plt.xlabel(r'$T$ [dimensionless units]')  # Temperature axis
plt.legend([r'$N_h = 8$', r'$N_h = 16$', r'$N_h = 256$'], loc='upper right')  # Legend for different RBM sizes
plt.savefig("cRBM.pdf")
plt.show()

# Plot average magnetization vs temperature
for i in range(len(numberHiddenNodes)):
    plt.plot(tempList, magnetisation[i], colours[i])
plt.ylabel(r'$\langle M \rangle$ [dimensionless units]')  # Magnetization label
plt.xlabel(r'$T$ [dimensionless units]')  # Temperature axis
plt.legend([r'$N_h = 8$', r'$N_h = 16$', r'$N_h = 256$'], loc='upper right')  # Legend
plt.savefig("magRBM.pdf")
plt.show()

# Plot average energy vs temperature
for i in range(len(numberHiddenNodes)):
    plt.plot(tempList, energy[i], colours[i])
plt.ylabel(r'$\langle E \rangle$ [dimensionless units]')  # Energy label
plt.xlabel(r'$T$ [dimensionless units]')  # Temperature axis
plt.legend([r'$N_h = 8$', r'$N_h = 16$', r'$N_h = 256$'], loc='lower right')  # Legend
plt.savefig("eRBM.pdf")
plt.show()


In [ ]:
# Initialize lists to store observables (magnetization, energy) and their variances
magnetisationVariance = []  # Variance of magnetization for each temperature
energyVariance = []  # Variance of energy for each temperature
magnetisation = []  # Average magnetization per spin for each temperature
energy = []  # Average energy per spin for each temperature

'''
Train an RBM for each temperature and compute physical observables using generated samples.
This process is repeated for different numbers of hidden nodes in the RBM.
'''
numberHiddenNodes = [8, 16, 256]  # Different RBM configurations with varying numbers of hidden nodes
numberHiddenNodes = [5]  # Different RBM configurations with varying numbers of hidden nodes


for h in range(len(numberHiddenNodes)):
    # Temporary storage for observables corresponding to this RBM configuration
    magnetisationOneTemperature = []
    energyOneTemperature = []
    magnetisationVarianceOneTemperature = []
    energyVarianceOneTemperature = []
    temperatureCounter = 0  # Index to track temperature in loops

    # Define a subset of temperatures for Gibbs sampling
    selected_temps = tempList[::20]  # Example: Sample every 5th temperature
    print(f"Selected temperatures for Gibbs sampling: {selected_temps}")

    for latticeOneTemperature in latticesTotal:
        temperature = temperatureLabels[temperatureCounter, 0]  # Extract current temperature

        if temperature in selected_temps:  # Only run Gibbs sampling for selected temperatures
            print(f"Running Gibbs sampling for T = {temperature}")

            # Gibbs sampling code goes here
            magnetisationSampleList = []
            energySampleList = []
            randomlattice = np.random.randint(2, size=latticeSize ** 2)

            equilibrationTime = 1000
            numberSamples = 1000

            for i in range(equilibrationTime * latticeSize ** 2 + numberSamples * latticeSize ** 2):
                lattice = rbm.gibbs(randomlattice)
                if i > equilibrationTime * latticeSize ** 2 and i % latticeSize ** 2 == 0:
                    magnetisationSampleList.append(LatticeMagnetisation(lattice, latticeSize) / latticeSize ** 2)
                    energySampleList.append(LatticeEnergy(lattice, latticeSize))

            # Store results as usual
            magnetisationOneTemperature.append(np.average(magnetisationSampleList))
            energyOneTemperature.append(np.average(energySampleList) / latticeSize ** 2)
            magnetisationVarianceOneTemperature.append((latticeSize ** 2 / temperature) * np.var(magnetisationSampleList))
            energyVarianceOneTemperature.append((1 / (temperature ** 2 * latticeSize ** 2)) * np.var(energySampleList))

        else:
            print(f"Skipping Gibbs sampling for T = {temperature}")

        temperatureCounter += 1  # Move to the next temperature

    # Store the computed observables for this RBM configuration
    magnetisation.append(magnetisationOneTemperature)
    energy.append(energyOneTemperature)

    magnetisationVariance.append(magnetisationVarianceOneTemperature)
    energyVariance.append(energyVarianceOneTemperature)


In [ ]:
# Define colors for plotting different RBM configurations
colours = ['ro', 'go', 'bo']
colours = ['go']

# Plot specific heat (energy variance) vs temperature
for i in range(len(numberHiddenNodes)):
    plt.plot(selected_temps, energyVariance[i], colours[i])
plt.ylabel("c [dimensionless units]")  # Specific heat label
plt.xlabel(r'$T$ [dimensionless units]')  # Temperature axis
plt.legend([r'$N_h = 8$', r'$N_h = 16$', r'$N_h = 256$'], loc='upper right')  # Legend for different RBM sizes
plt.savefig("cRBM.pdf")
plt.show()

# Plot average magnetization vs temperature
for i in range(len(numberHiddenNodes)):
    plt.plot(selected_temps, magnetisation[i], colours[i])
plt.ylabel(r'$\langle M \rangle$ [dimensionless units]')  # Magnetization label
plt.xlabel(r'$T$ [dimensionless units]')  # Temperature axis
plt.legend([r'$N_h = 8$', r'$N_h = 16$', r'$N_h = 256$'], loc='upper right')  # Legend
plt.savefig("magRBM.pdf")
plt.show()

# Plot average energy vs temperature
for i in range(len(numberHiddenNodes)):
    plt.plot(selected_temps, energy[i], colours[i])
plt.ylabel(r'$\langle E \rangle$ [dimensionless units]')  # Energy label
plt.xlabel(r'$T$ [dimensionless units]')  # Temperature axis
plt.legend([r'$N_h = 8$', r'$N_h = 16$', r'$N_h = 256$'], loc='lower right')  # Legend
plt.savefig("eRBM.pdf")
plt.show()